In [ ]:
# Chains in LangChain

## Connect to OpenAI and define our model

Create an OpenAI API key for this project at [https://platform.openai.com/account/api-keys](https://platform.openai.com/account/api-keys). 

Copy `.env.sample` to `.env` and replace `<YOUR_OPENAI_API_KEY>` with the one you created.

Let's get started!

Let's make sure that we can read our OpenAI key from the `.env` file we created.

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Specify our model
llm_model="gpt-3.5-turbo-1106"

## Load our example CSV file using pandas

Before we begin, let's install the following dependencies from the command-line of our Python virtual environment:

```sh
(.venv) % pip install pandas
(.venv) % pip freeze > requirements.txt
```

In [5]:
import pandas as pd
df = pd.read_csv('../data/Data.csv')

VERIFY: Make sure we have some data loaded into our dataframe

In [6]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [8]:
# Initialize the language model we want to use with a high temperature to have creative description
llm = ChatOpenAI(temperature=0.9, model=llm_model)

/Users/rob/repos/deeplearning-ai/langchain-for-llm-application-development/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [9]:
# Our prompt
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [10]:
# Combine the LLM and the prompt into a chain
chain = LLMChain(llm=llm, prompt=prompt)

In [14]:
# Use the chain to come up with a name for a company that makes queen size sheet sets
product = "Queen Size Sheet Set"
chain.run(product)

'"Regal Linens"'

Experiment with different products or re-run the chain to see what responses get generated.

This will work with products that do not exist in the source CSV file. Give it a shot. 🤓

## SimpleSequentialChain

SimpleSequentialChain's are useful when you need to serially execute chains - typically chains that have one input and one output.

In the example below:

- Our first chain will take in the product name defined above - "Queen Size Sheet Set"
  - The output will be a creative name for a company to make the product - queen size sheet sets in our case
- Our second chain will receive the creative name for a company and write a creative twenty word description for the company name

In [15]:
from langchain.chains import SimpleSequentialChain

In [16]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [17]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [18]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [19]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
"Royal Dreams Linens"
Royal Dreams Linens offers luxurious and elegant bedding and home decor, providing customers with the ultimate in comfort and style.

> Finished chain.


'Royal Dreams Linens offers luxurious and elegant bedding and home decor, providing customers with the ultimate in comfort and style.'

## SequentialChain

## Router Chain